# 安装依赖

In [ ]:
!git clone https://github.com/HongzheBi/DocQA.git

Cloning into 'DocQA'...
remote: Enumerating objects: 249, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 249 (delta 32), reused 49 (delta 18), pack-reused 180
Receiving objects: 100% (249/249), 718.31 MiB | 17.66 MiB/s, done.
Resolving deltas: 100% (45/45), done.


In [ ]:
!cd DocQA/finetune/ && pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.6/416.6 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 119.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 90.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 125.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━

# LoRA微调

In [ ]:
!mkdir sft_checkpoint
!mkdir output

In [4]:
!CUDA_VISIBLE_DEVICES=0 
!cd DocQA/finetune/src/ && python train_sft.py \
    --do_train \
    --dataset webqa \
    --finetuning_type p_tuning \
    --output_dir /content/sft_checkpoint \
    --per_device_train_batch_size 8 \
    --gradient_accumulation_steps 4 \
    --lr_scheduler_type cosine \
    --logging_steps 10 \
    --save_steps 1000 \
    --learning_rate 2e-2 \
    --num_train_epochs 3.0 \
    --pre_seq_len 128 \
    --prefix_projection False \
    --fp16

2023-05-21 11:48:29.450838: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
05/21/2023 11:48:32 - WARNING - utils.common - Process rank: 0, device: cuda:0, n_gpu: 1
  distributed training: True, 16-bits training: True
05/21/2023 11:48:32 - INFO - utils.common - Training/evaluation parameters Seq2SeqTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=True,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, '

In [5]:
!cd DocQA/finetune/src/ && python export_model.py \
    --checkpoint_dir /content/sft_checkpoint \
    --output_dir /content/output

2023-05-21 14:37:19.815925: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
05/21/2023 14:37:22 - INFO - utils.common - Load fine-tuned model from checkpoint(s): /content/sft_checkpoint
A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm-6b:
- tokenization_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
Loading checkpoint shards: 100% 8/8 [00:10<00:00,  1.36s/it]
Some weights of ChatGLMForConditionalGeneration were not initialized from the model checkpoint at THUDM/chatglm-6b and are newly initialized: ['transformer.prefix_encoder.embedding.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
05/21/2023 14:37:34 - INFO - utils.common - Fine-tuning method: P-Tuning v2
trainable params: 0 || all params: 6202646528 || train

In [6]:
!CUDA_VISIBLE_DEVICES=0 
!cd DocQA/finetune/src/ && python train_sft.py \
    --do_eval \
    --dataset webqa \
    --checkpoint_dir /content/sft_checkpoint \
    --output_dir /content/output \
    --per_device_eval_batch_size 2 \
    --max_samples 50 \
    --predict_with_generate

2023-05-21 14:38:16.190267: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
05/21/2023 14:38:18 - WARNING - utils.common - Process rank: 0, device: cuda:0, n_gpu: 1
  distributed training: True, 16-bits training: False
05/21/2023 14:38:18 - INFO - utils.common - Training/evaluation parameters Seq2SeqTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0,